# 뉴스 요약(JSON) 생성 설계 설명서

## 1. 목적

크롤링된 뉴스 기사 본문을 짧은 요약문으로 정리하여 JSON 파일로 저장한다.    
이 요약은 이후의 검색, 질의응답, 핵심 이슈 탐지, 대시보드 구축 등 후속 작업의 효율을 높이는 전처 단계로 사용된다.

## 2. 요약을 진행 한 이유
1. 비용과 속도의 절감
* 앞으로 추출하게될 `핵심 이슈`, `위치 정보`, `핵심 인물` 을 추출하는데 있어서 본문을 모두 탐색하기보단 요약된 정보를 활용해보기 위해 Summary를 생성
* 본문 사용시 토큰 사용량이 커지고 응답 속도도 느려진다는 단점이 있기 때문에 요약문을 사용한다면 보완할 수 있다.
* LLM을 활용하게 된다면 문서 당 3~4문장의 요약만으로도 핵심 맥락 파악이 가능해 LLM 호출 비용과 응답 시간을 줄일 수 있다.

2. 탐색과 필터링의 효율
* 수천 건의 기사 중에서 핵심만 빠르게 훑어보기 위해서 짧은 요약 목록도 필요하다.
* 요약문을 활용하면 관심 기사 선별이 쉬워진다.

3. 후속 분석의 품질 향상
* 사건 추출, Top10 이슈 랭킹, 키워드 클러스터링 등에서 핵심 표현이 응축된 요약을 사용하면    
군더더기 문구(저작권, 구독 문안, 이메일 등)에 덜 영향을 받고 신호 대 잡음비가 높아진다.

## 3. 활용 가능성

* 이슈 모니터링 대시보드 : 기간 및 키워드별 기사 목록에서 한 줄 요약을 제공하여 가독성을 향상시킬 수 있다.
* 사건 랭킹과 정합성 점검 : 사건성 구 추출 결과와 요약문을 나란히 확인하여 의미 정확도를 검증해볼 수 있다.
* 일간, 주간 등 리포트에서 기사 원문이 아닌 요약본을 활용해 전달하는것을 고려해 볼 수 있다.

## 4. 설계 근거
1. 원문 보존과 신뢰성
* 코드가 본문에서 중요 문장만 발췌하는 방식을 사용한다.
* 생성형 요약처럼 원문에 없는 말을 덧붙이지 않아 사실 왜곡 가능성이 낮다.

2. 운영 안정성
* 한국어 문자 임베딩 기반의 수치적 기준 (코사인 유사도)으로 문장을 선택한다.
* 모델·프로프트에 덜 민감해 일관된 결과를 얻을 수 있다.

3. 유연한 확장
* 요약 문자 수는 파라미터로 조정이 가능하다.
* 필요 시 문장·전처리·정규식만 바꿔도 도메인 특성에 맞춘 튜닝이 가능하다.

## 5. 구현 방법

1.  전처리
- 불필요 문구 제거: 저작권 문구, 구독 유도, “영상 닫기”, 대괄호 문구 등 반복 노이즈 패턴을 정규식으로 제거    
(예: `ⓒ 연합뉴스`, `무단전재 및 재배포 금지`, `이미지 확대`, `기사 입력`, `송고 시간`, `[...]` 등)
- 기자·이메일 처리:
    - 기자 이름 패턴(한글·영문, “기자·특파원·통신원·논설위원·평론가” 접미 포함)과
    - 이메일 주소를 검출하여 요약문에서 제거
- 공백·제어문자 정리: 숨은 공백과 특수 공백을 제거하고 띄어쓰기를 정돈

1.  문장 분할
- 우선 KSS 라이브러리로 한국어 문장 분할을 수행하고, 
실패 시 구두점 기반 대체 분할 규칙으로 문장을 나눈다.

1. 문장 임베딩과 중요도 계산
- 임베딩 모델: `MODEL_NAME = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"`    
(sentence-transformers 한국어 SBERT 계열 모델)
- 각 문장의 임베딩 벡터를 구한 뒤 문서 임베딩(문장 임베딩 평균) 과의 코사인 유사도를 계산한다.
- 유사도가 높은 순서대로 상위 문장을 고른다.
- 코드 기본값은 3문장이지만, 실행부에서는 4문장 요약을 사용했다.    
(필요 시 `num_sentences`로 조정 가능)

1. 요약 후 정리
- 선택된 문장들을 원문 순서대로 이어 붙여 요약문을 만든다.
- 전처리 단계에서 수집한 기자 이름·이메일을 다시 한 번 제거하여 요약에 남지 않도록 한다.
- 각 기사 딕셔너리에 `summary`를 추가하여 리스트로 재저장한다.
- 저장은 `ensure_ascii=False`, `indent=4`로 진행하여 한글 보존과 가독성을 확보한다.

## 6. 파라미터와 기본값

- 요약 문장 수: 기본 3문장, 현재 실행은 4문장
- 임베딩 모델: `snunlp/KR-SBERT-V40K-klueNLI-augSTS`
- 문장 분할: KSS 우선, 실패 시 대체 규칙
- 난수 시드: 42(결과 재현성 확보)
- 디바이스: GPU가 있으면 GPU, 없으면 CPU 자동 선택

## 7. ## 산출물 예시

```json
{
  "title": "예시 기사 제목",
  "text": "기사 원문 전체...",
  "url": "https://example.com",
  "pubDate": "2024-06-01",
  "category": "정치",
  "summary": "핵심 문장 1. 핵심 문장 2. 핵심 문장 3. 핵심 문장 4."
}
```


## 8. 품질관리 포인트
* 잡음 제거 목록을 별도 상수로 관리하여, 필요 시 문구를 추가·수정한다.
* 기자명·이메일은 이름 경계를 고려한 정규식으로 처리해 과도한 삭제를 피한다.
* 입력 JSON의 구조가 일정하지 않을 수 있으므로, 결측·타입 검증을 수행한다.
* 문장 수가 매우 적은 기사(짧은 속보 등)는 본문 전체를 요약으로 인정하여 빈 요약을 최소화한다.

## 9. 한계와 보완 방향

1. 추출 요약 특성상 문장 재배열·표현 압축은 제한적이다.    
설명적·자유로운 한 문장 요약이 필요하면 생성형 요약을 보조적으로 결합할 수 있다.    
    
2. 사건·장소·인물과 같은 구조적 필드를 요약 단계에서 함께 추출하지는 않는다.    
필요 시 NER·규칙 기반 추출을 요약 이후 단계에 붙여 확장할 수 있다.    
    
3. 매우 긴 기사에서 상위 3~4문장만으로는 맥락이 부족할 수 있다.    
이 경우 문장 수를 상황별로 조정하거나 섹션별 요약을 고려한다.    